In [4]:
import numpy as np
import matplotlib.pyplot as plt


def soft_thresholding(x, kappa):
    return np.sign(x) * np.maximum(np.abs(x) - kappa, 0)


def admm_sparse_regression(A, b, lam, rho, tol=1e-5, max_iter=1800):
    m, n = A.shape
    x = np.zeros(n)
    z = np.zeros((m, 1))
    u = np.zeros((m, 1))

    AtA = A.T @ A
    I = np.identity(n)
    Atb = A.T @ b

    for iteration in range(max_iter):
        # Update x
        x = np.linalg.inv(AtA + rho * I) @ (Atb + A.T @ (z - u))
        # Update u
        u += rho * (A @ x - b + z) / (1 + rho)
        # Store old z to compute dual feasibility
        z_old = z.copy()
        # Update z
        z += rho * (A @ x + u - b)
        # Compute residuals
        r = A @ x + u - b  # Primal feasibility
        s = rho * A.T @ (z - z_old)  # Dual feasibility

        Rp = np.linalg.norm(r, np.inf) / (1 + np.linalg.norm(b, np.inf))
        Rd = np.linalg.norm(s, np.inf) / (1 + np.linalg.norm(A.T @ b, np.inf))

        if Rp < tol and Rd < tol:
            break

    return x, iteration


def plot_results(x_true, x_est):
    plt.figure(figsize=(10, 5))
    plt.stem(
        x_true.flatten(),
        linefmt="r-",
        markerfmt="ro",
        basefmt="k-",
        label="True x (Red Circles)",
    )
    plt.stem(
        x_est,
        linefmt="b-",
        markerfmt="b*",
        basefmt="k-",
        label="Estimated x (Blue Stars)",
    )
    plt.title("Comparison of True and Estimated Solutions")
    plt.xlabel("Index")
    plt.ylabel("Value")
    plt.legend()
    plt.show()


# Data setup
np.random.seed(0)
m, n = 500, 2000
x_true = np.ceil(2 * np.random.randn(n, 1) * 0.02)
A = np.random.rand(m, n)
noise = np.random.rand(m, 1)
noise /= np.linalg.norm(noise)
b = A @ x_true + 0.1 * np.linalg.norm(A @ x_true) * noise

lam_max = np.max(np.abs(A.T @ b))
lam = 0.001 * lam_max
rho = 1.0

x_est, num_iterations = admm_sparse_regression(A, b, lam, rho)
print(f"Estimated x with {num_iterations} iterations.")
plot_results(x_true, x_est)